## Using structured outputs

Gemnini supports pydantic.

In [1]:
#%pip install --upgrade --quiet pydantic-ai

In [2]:
# Needed in Jupyter environment See: https://ai.pydantic.dev/troubleshooting/ 
import nest_asyncio

nest_asyncio.apply()

In [3]:
MODEL_ID = "gemini-1.5-flash"

import os
from dotenv import load_dotenv
load_dotenv("../keys.env")
assert os.environ["GEMINI_API_KEY"][:2] == "AI",\
       "Please specify the GEMINI_API_KEY access token in keys.env file"

In [4]:
import pydantic_ai
from pydantic_ai.models.gemini import GeminiModel

model = GeminiModel(MODEL_ID, api_key=os.getenv('GEMINI_API_KEY'))

## Structured data generation

Specifying the structure of the data we want

In [5]:
from dataclasses import dataclass
from enum import Enum

class CurrencyEnum(str, Enum):
    USD = 'USD'
    UKP = 'UKP'
    INR = 'INR'
    EUR = 'EUR'

@dataclass
class Invoice:
    purpose: str
    amount: float
    currency: CurrencyEnum = CurrencyEnum.USD

In [6]:
from pydantic_ai import Agent

def parse_invoice_info(paragraph: str) -> str:
    system_prompt = """
    You will be given a short snippet from an email that represents an invoice.
    Extract the purpose and amount of the invoice.
    """
    
    agent = Agent(model,
                  result_type=Invoice,
                  system_prompt=system_prompt)

    response = agent.run_sync(paragraph)
    return response.data

result = parse_invoice_info("""
Requesting reimbursement for taxi-ride to airport. I paid $32.30.
""")
print(result)

Invoice(purpose='taxi-ride to airport', amount=32.3, currency=<CurrencyEnum.USD: 'USD'>)


In [7]:
result = parse_invoice_info("""
I spent €43 entertaining our French clients.
""")
print(result)

Invoice(purpose='entertaining French clients', amount=43.0, currency=<CurrencyEnum.EUR: 'EUR'>)


In [8]:
result = parse_invoice_info("""
My dinner at the airport cost ₹820
""")
print(result)

Invoice(purpose='Dinner at the airport', amount=820.0, currency=<CurrencyEnum.INR: 'INR'>)
